In [2]:
import plotly.graph_objects as go

def create_component_diagram_plotly():
    fig = go.Figure()

    # --- Определение позиций компонентов (ручное позиционирование) ---
    components = {
        "Frontend": {"x": 0, "y": 10, "label": "Frontend (Mobile/Web App)"},
        "Backend": {"x": 5, "y": 10, "label": "Backend (Bank's System)"},
        "Integration": {"x": 10, "y": 10, "label": "Integration Layer"},
        "CBRPlatform": {"x": 15, "y": 10, "label": "CBR Digital Ruble Platform"},
        "OperDB": {"x": 5, "y": 5, "label": "Operational DBs (PostgreSQL, Redis)", "is_db": True},
        "AnalytDB": {"x": 10, "y": 5, "label": "Analytical DBs (ClickHouse)", "is_db": True},
        "Analytics": {"x": 10, "y": 0, "label": "Analytical Reporting Layer"},
    }

    # Размеры блоков
    component_width = 3.5
    component_height = 2.0

    # --- Добавление компонентов (прямоугольников) и текста ---
    for name, props in components.items():
        x0, y0 = props["x"], props["y"]
        x1, y1 = x0 + component_width, y0 + component_height

        # Отрисовка прямоугольника компонента
        fig.add_shape(
            type="rect",
            x0=x0, y0=y0, x1=x1, y1=y1,
            line=dict(color="RoyalBlue", width=2),
            fillcolor="LightSkyBlue" if not props.get("is_db", False) else "LightGrey",
            layer="below"
        )

        # Отрисовка символа базы данных для DBs (ОСТАВЛЯЕМ ТОЛЬКО ПРЯМОУГОЛЬНИК)
        if props.get("is_db", False):
            # Простая имитация символа DB - только верхний прямоугольник
            fig.add_shape(
                type="rect",
                x0=x0 + component_width * 0.1, y0=y1 - component_height * 0.1,
                x1=x1 - component_width * 0.1, y1=y1 + component_height * 0.1, # Простирается немного выше основного блока
                line=dict(color="RoyalBlue", width=2),
                fillcolor="LightGrey",
                layer="below"
            )
            # УДАЛИТЕ СТРОКИ, КОТОРЫЕ ПЫТАЛИСЬ НАРИСОВАТЬ ЭЛЛИПС:
            # fig.add_shape(
            #     type="ellipse",
            #     xref="x", yref="y",
            #     x0=x0 + component_width * 0.1, y0=y1 - component_height * 0.1 - 0.2,
            #     x1=x1 - component_width * 0.1, y1=y1 + component_height * 0.1 - 0.2,
            #     line_color="RoyalBlue", fillcolor="LightGrey", opacity=1, layer="below"
            # )


        # Добавление текста (метки компонента)
        fig.add_annotation(
            x=(x0 + x1) / 2, y=(y0 + y1) / 2,
            text=props["label"],
            showarrow=False,
            font=dict(size=10, color="black"),
            align="center",
            valign="middle"
        )

    # --- Добавление стрелок (соединений) ---
    # Для простоты, координаты стрелок также задаются вручную или вычисляются
    # Это наиболее трудоемкая часть и требует точных расчетов для аккуратности

    connections = [
        # (source_comp, target_comp, label, from_side, to_side)
        ("Frontend", "Backend", "Requests & Data", "right", "left"),
        ("Backend", "OperDB", "DB Calls", "bottom", "top"),
        ("Backend", "Integration", "API Calls", "right", "left"),
        ("Integration", "CBRPlatform", "Digital Ruble Operations", "right", "left"),
        ("Analytics", "OperDB", "ETL", "top", "bottom"),
        ("Analytics", "AnalytDB", "DB Calls", "top", "top"), # Сложная линия
        ("AnalytDB", "Analytics", "Data for Reports", "bottom", "bottom") # Сложная линия
    ]

    for conn in connections:
        source_name, target_name, label, from_side, to_side = conn
        source = components[source_name]
        target = components[target_name]

        # Вычисление точек для стрелок
        # Это очень упрощенные вычисления. Для реальной диаграммы нужно гораздо больше логики.
        # Например, для "right" это x1, для "bottom" это y0, и т.д.
        # Точное позиционирование коннекторов на сторонах компонента
        x_src, y_src = source["x"], source["y"]
        x_tgt, y_tgt = target["x"], target["y"]

        # Упрощенные точки для соединения центров для прямолинейных связей
        # Для более сложных связей (как для DBs) нужно вычислять точки на границах прямоугольников
        x_start, y_start = x_src + component_width / 2, y_src + component_height / 2
        x_end, y_end = x_tgt + component_width / 2, y_tgt + component_height / 2

        # Смещение для более красивых прямых линий между центрами
        if source_name == "Frontend" and target_name == "Backend":
            x_start = source["x"] + component_width # Правая сторона Frontend
            y_start = source["y"] + component_height / 2
            x_end = target["x"] # Левая сторона Backend
            y_end = target["y"] + component_height / 2
        elif source_name == "Backend" and target_name == "OperDB":
            x_start = source["x"] + component_width / 2
            y_start = source["y"] # Нижняя сторона Backend
            x_end = target["x"] + component_width / 2
            y_end = target["y"] + component_height # Верхняя сторона OperDB
        elif source_name == "Backend" and target_name == "Integration":
            x_start = source["x"] + component_width # Правая сторона Backend
            y_start = source["y"] + component_height / 2
            x_end = target["x"] # Левая сторона Integration
            y_end = target["y"] + component_height / 2
        elif source_name == "Integration" and target_name == "CBRPlatform":
            x_start = source["x"] + component_width # Правая сторона Integration
            y_start = source["y"] + component_height / 2
            x_end = target["x"] # Левая сторона CBRPlatform
            y_end = target["y"] + component_height / 2
        elif source_name == "Analytics" and target_name == "OperDB":
            x_start = source["x"] + component_width / 2
            y_start = source["y"] + component_height # Верхняя сторона Analytics
            x_end = target["x"] + component_width / 2
            y_end = target["y"] # Нижняя сторона OperDB
        elif source_name == "Analytics" and target_name == "AnalytDB":
            # Особый случай для отрисовки линии вправо, затем вниз
            x_start = source["x"] + component_width
            y_start = source["y"] + component_height / 2
            x_mid = target["x"] + component_width / 2
            y_mid = target["y"] + component_height
            # Отрисовка двух сегментов для ломаной линии
            fig.add_annotation(
                ax=x_start, ay=y_start,
                x=x_mid, y=y_start, # Горизонтальный сегмент
                axref="x", ayref="y", xref="x", yref="y",
                showarrow=True, arrowhead=2, arrowsize=1, arrowwidth=1, arrowcolor="black",
                text="", font=dict(size=8), bordercolor="black", borderwidth=0
            )
            fig.add_annotation(
                ax=x_mid, ay=y_start, # От конца горизонтального сегмента
                x=x_mid, y=y_mid, # Вертикальный сегмент до середины верхней границы AnalytDB
                axref="x", ayref="y", xref="x", yref="y",
                showarrow=True, arrowhead=2, arrowsize=1, arrowwidth=1, arrowcolor="black",
                text="", font=dict(size=8), bordercolor="black", borderwidth=0
            )
            fig.add_annotation(
                x=(x_start + x_mid) / 2, y=y_start + 0.2, # Метка на горизонтальном сегменте
                text=label,
                showarrow=False,
                font=dict(size=8),
                align="center",
                valign="middle"
            )
            continue # Пропускаем стандартную отрисовку для этой связи

        elif source_name == "AnalytDB" and target_name == "Analytics":
            # Особый случай для отрисовки линии вверх, затем влево
            x_start = source["x"] + component_width / 2
            y_start = source["y"] + component_height # От верхней стороны AnalytDB
            x_mid = source["x"] + component_width / 2 # Вертикальный сегмент
            y_mid = target["y"] + component_height / 2 # До середины левой стороны Analytics
            x_end = target["x"] + component_width # До правой стороны Analytics
            y_end = target["y"] + component_height / 2

            # Отрисовка двух сегментов для ломаной линии
            fig.add_annotation(
                ax=x_start, ay=y_start,
                x=x_mid, y=y_mid, # Вертикальный сегмент
                axref="x", ayref="y", xref="x", yref="y",
                showarrow=True, arrowhead=2, arrowsize=1, arrowwidth=1, arrowcolor="black",
                text="", font=dict(size=8), bordercolor="black", borderwidth=0
            )
            fig.add_annotation(
                ax=x_mid, ay=y_mid,
                x=x_end, y=y_end, # Горизонтальный сегмент
                axref="x", ayref="y", xref="x", yref="y",
                showarrow=True, arrowhead=2, arrowsize=1, arrowwidth=1, arrowcolor="black",
                text="", font=dict(size=8), bordercolor="black", borderwidth=0
            )
            fig.add_annotation(
                x=(x_mid + x_end) / 2, y=y_mid + 0.2, # Метка на горизонтальном сегменте
                text=label,
                showarrow=False,
                font=dict(size=8),
                align="center",
                valign="middle"
            )
            continue # Пропускаем стандартную отрисовку для этой связи


        # Добавление стрелки
        fig.add_annotation(
            x=x_end, y=y_end,
            ax=x_start, ay=y_start,
            xref="x", yref="y", axref="x", ayref="y",
            showarrow=True,
            arrowhead=2,  # Тип стрелки
            arrowsize=1,
            arrowwidth=1,
            arrowcolor="black",
            text=label,
            font=dict(size=8),
            # Смещение текста для метки
            textangle=0,
            xanchor="center", yanchor="middle"
        )


    fig.update_layout(
        title="Архитектура данных цифрового рубля (имитация UML)",
        xaxis=dict(visible=False, range=[-2, 18]), # Подгонка диапазона осей
        yaxis=dict(visible=False, range=[-2, 12]),
        showlegend=False,
        hovermode=False,
        height=600, # Высота графика
        width=1000, # Ширина графика
        margin=dict(l=20, r=20, t=50, b=20)
    )

    fig.show()

create_component_diagram_plotly()